In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import timescaledb_model as tsdb
from analyze import get_files_infos_df, read_file, get_files_infos_windows_df
from tqdm import tqdm 
from utils import multi_read_df_from_paths
from constant import DATA_PATH, IS_DOCKER
import os

In [3]:
files_info_path = os.path.join(DATA_PATH, 'files_infos.pkl')
# files_infos_df = get_files_infos_df()
# files_infos_df = get_files_infos_windows_df()  # for windows
# files_infos_df.to_pickle(files_info_path)

In [4]:
files_infos_df = get_files_infos_df(files_info_path)
# files_infos_df = get_files_infos_windows_df(files_info_path) #for windows

In [5]:
np.random.seed(10)
dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
file_infos_df_filtered = files_infos_df[files_infos_df["date"].isin(dates_to_take)]
file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour
file_infos_df_filtered = file_infos_df_filtered.groupby([file_infos_df_filtered["date"], file_infos_df_filtered["hour"], file_infos_df_filtered["market"]]).first().reset_index()

/var/folders/lb/9z4xr9hd1013w3_cbsw1p6600000gn/T/ipykernel_14883/2876105170.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dates_to_take = files_infos_df.groupby([files_infos_df["year_month"]]).apply(lambda x :x.sample(n=1)).reset_index(drop=True)["date"]
/var/folders/lb/9z4xr9hd1013w3_cbsw1p6600000gn/T/ipykernel_14883/2876105170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_infos_df_filtered["hour"] = file_infos_df_filtered.index.hour


## DB

In [6]:
import timescaledb_model as tsdb
db = (
    tsdb.TimescaleStockMarketModel("bourse", "ricou", "db", "monmdp", setup=True)
    if IS_DOCKER
    else tsdb.TimescaleStockMarketModel("bourse", "ricou", "localhost", "monmdp", setup=True)
)

Logs of timescaledb_model go to ../data/bourse.log


# Utils

In [7]:
import re

def is_possible_ticker_with_number(symbol):
    return bool(re.match(r'^[A-Z0-9\-]{1,10}$', symbol))

def detection_intrus(df: pd.DataFrame):
    return df[(df["ticker"].apply(is_possible_ticker_with_number) == False)]

In [40]:
def get_prefix(df: pd.DataFrame, prefix_function = lambda x : x[0:3]) -> set:
    return set(df["symbol"].apply(prefix_function).unique())

In [9]:
map_prefix_to_symbol_nf = {
    "1rP": lambda x: x[3:].split("_")[0] if len(x) != 15 else np.nan,  # EuroNext Pars
    "1rA": lambda x: x[3:],  # EuroNext Amsterdam
    "1rE": lambda x: x[4:],  # EuroNext Paris
    "FF1": lambda x: x.split("_")[1][0:],  # EuroNext Brussels
}


def update_ticker_column(
    df: pd.DataFrame,
) -> pd.DataFrame:
    df["ticker"] = df.apply(
        lambda x: map_prefix_to_symbol_nf.get(x["prefix"], lambda _: x["symbol"])(
            x["symbol"]
        ),
        axis=1,
    )
    return df

def update_mid_column(
    df: pd.DataFrame,
    prefix_to_market_id: dict,
    default_mid: int,
) -> pd.DataFrame:
    df["mid"] = df.apply(
        lambda x: prefix_to_market_id.get(x["prefix"],default_mid),
        axis=1,
    )
    return df

In [10]:
def dfs_to_companie(
    dfs: list[pd.DataFrame], prefix_to_market_id: dict, default_mid: int, is_pea: bool = False
) -> pd.DataFrame:
    df_all_days = pd.concat(dfs)
    df_all_days_grouped = (
        df_all_days.reset_index(drop=True).groupby(["symbol", "name"]).last()
    )
    df_all_days_grouped["name"] = df_all_days_grouped.index.get_level_values(1)
    df_companies = df_all_days_grouped.groupby(
        df_all_days_grouped.index.get_level_values(0)
    ).last()
    df_companies = df_companies.reset_index()[["symbol", "name", "timestamp"]]
    df_companies["prefix"] = df_companies["symbol"].apply(lambda x: x[0:3])
    df_companies = update_ticker_column(df_companies)
    df_companies = update_mid_column(df_companies, prefix_to_market_id, default_mid=default_mid)
    df_companies = df_companies.drop(columns=["prefix"])
    df_companies["pea"] = is_pea
    return df_companies

## Amsterdam - Companies handling

In [11]:
dfs_amsterdam = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "amsterdam"] ["path"]))

100%|██████████| 540/540 [00:11<00:00, 47.71it/s]


In [12]:
amsterdam_companies = dfs_to_companie(dfs_amsterdam, db.prefix_to_market_id, default_mid=db.nasdaq_market_id) 

In [13]:
amsterdam_companies

,symbol,name,timestamp,ticker,mid,pea
0,124718367,ASTRAZENECA FIN CVR,2023-03-16 17:02:01,124718367,14,False
1,AABA,ALTABA,2019-10-28 17:02:02,AABA,14,False
2,AACG,ATA CRTV GLB SP ADR,2023-12-21 17:02:02,AACG,14,False
3,AACI,ARMADA ACQN I,2023-12-21 17:02:02,AACI,14,False
4,AACIU,ARMADA ACQN I UTS,2023-12-21 17:02:02,AACIU,14,False
...,...,...,...,...,...,...
6684,ZWRKU,Z-WORK ACQN UTS 26,2022-12-20 17:02:01,ZWRKU,14,False
6685,ZY,ZYMERGEN,2022-11-18 17:02:02,ZY,14,False
6686,ZYME,ZYMEWORKS,2023-12-21 17:02:02,ZYME,14,False
6687,ZYNE,ZYNERBA PHARMA,2023-10-30 17:02:01,ZYNE,14,False


In [14]:
# detection_intrus(amsterdam_companies)

## CompA - Companies handling

In [15]:
dfs_compA = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "compA"] ["path"]))

100%|██████████| 540/540 [00:02<00:00, 189.91it/s]


In [41]:
compA_companies = dfs_to_companie(dfs_compA, prefix_to_market_id=db.prefix_to_market_id ,default_mid=db.prefix_to_market_id["1rP"])

In [42]:
compA_companies

,symbol,name,timestamp,ticker,mid,pea
0,1rAAF,AIR FRANCE - KLM,2023-12-21 17:02:02,AF,11,False
1,1rAAFA,AIR FRANCE - KLM,2023-12-21 17:02:02,AFA,11,False
2,1rAENX,EURONEXT,2023-12-21 17:02:02,ENX,11,False
3,1rAGTO,GEMALTO,2019-06-21 17:02:02,GTO,11,False
4,1rARDSADT,ROYAL D SH DRP 21,2021-10-05 17:02:01,RDSADT,11,False
...,...,...,...,...,...,...
404,FF11_ENX,EURONEXT,2023-12-21 17:02:02,ENX,13,False
405,FF11_FP,TOTALENERGIES,2021-08-04 17:02:02,FP,13,False
406,FF11_SEV,SUEZ,2023-12-21 17:02:02,SEV,13,False
407,FF11_SGO,SAINT-GOBAIN,2023-12-21 17:02:02,SGO,13,False


In [18]:
# detection_intrus(compA_companies)

In [19]:
get_prefix(compA_companies)

{'1rA', '1rP', 'FF1'}

## CompB - Companies handling

In [20]:
dfs_compB = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "compB"] ["path"]))

100%|██████████| 540/540 [00:04<00:00, 119.51it/s]


In [21]:
compB_companies = dfs_to_companie(dfs_compB, prefix_to_market_id=db.prefix_to_market_id ,default_mid=db.prefix_to_market_id["1rP"])

In [22]:
compB_companies

,symbol,name,timestamp,ticker,mid,pea
0,1rP03227,LATECOERE,2023-12-21 17:02:02,03227,12,False
1,1rP2MX,TERACT R PFD,2023-12-21 17:02:02,2MX,12,False
2,1rP5478,CLARANOVA,2021-10-05 17:02:01,5478,12,False
3,1rPAAA,ALAN ALLMAN ASSOCIATES,2023-12-21 17:02:02,AAA,12,False
4,1rPAAC,ACCOR ACQ COM R PFD,2021-06-01 17:02:01,AAC,12,False
...,...,...,...,...,...,...
391,1rPXFAB,X-FAB SILICON,2023-12-21 17:02:02,XFAB,12,False
392,1rPXIL,XILAM ANIMATION,2023-12-21 17:02:02,XIL,12,False
393,1rPYSYT,SALVEPORN01JAN22EX,2023-12-21 17:02:02,YSYT,12,False
394,FF11_AKA,AKKA TECHNOLOGIES,2022-01-14 17:02:01,AKA,13,False


In [23]:
# get_prefix(compB_companies)

## PEA PME - Companies Handling

In [24]:
dfs_peapme = multi_read_df_from_paths(list(file_infos_df_filtered[file_infos_df_filtered["market"] == "peapme"] ["path"]))

  0%|          | 0/324 [00:00<?, ?it/s]

100%|██████████| 324/324 [00:02<00:00, 133.01it/s]


In [43]:
peapme_companies = dfs_to_companie(dfs_peapme, prefix_to_market_id=db.prefix_to_market_id ,default_mid=db.prefix_to_market_id["1rP"], is_pea=True)

In [44]:
peapme_companies

,symbol,name,timestamp,ticker,mid,pea
0,1rAADUX,ADUX,2023-12-21 17:02:02,ADUX,11,True
1,1rABESI,BESI,2023-12-21 17:02:02,BESI,11,True
2,1rEPALANT,ANTEVENIO,2021-11-16 17:02:01,ALANT,12,True
3,1rEPALAQU,AQUILA,2023-12-21 17:02:02,ALAQU,12,True
4,1rEPALBDM,BD MULTI MEDIA,2023-12-21 17:02:02,ALBDM,12,True
...,...,...,...,...,...,...
672,FF11_EVS,EVS BROADCAST EQU,2023-12-21 17:02:02,EVS,13,True
673,FF11_GKTX,GENKYOTEX,2022-08-08 17:02:02,GKTX,13,True
674,FF11_KIN,KINEPOLIS GROUP,2023-12-21 17:02:02,KIN,13,True
675,FF11_MLMAZ,MAZARO,2023-12-21 17:02:02,MLMAZ,13,True


In [27]:
get_prefix(peapme_companies)

{'1rA', '1rE', '1rP', 'FF1'}

## Join companies

In [45]:
df_companies = [amsterdam_companies, compA_companies, compB_companies, peapme_companies]

In [46]:
df_companies = pd.concat(df_companies)
df_companies.sort_values(by="timestamp", inplace=True)
df_companies.drop_duplicates(inplace=True)
df_companies.drop_duplicates(subset=["symbol"], keep="last", inplace=True)

### Check no duplicated

In [47]:
df_companies[df_companies.duplicated(subset=["symbol"], keep=False)][:50]

,symbol,name,timestamp,ticker,mid,pea


In [48]:
df_companies.drop("timestamp", axis=1, inplace=True)

In [49]:
df_companies.set_index("symbol", inplace=True)

In [50]:
df_companies.head()

,name,ticker,mid,pea
symbol,,,,
GOV,GV P IN REIT-SBI,GOV,14,False
BBOX,BLACK BOX,BBOX,14,False
INTX,INTERSECTIONS,INTX,14,False
FSACU,FEDRL STR ACQST UTS,FSACU,14,False
FSAC,FDRL STR ACQST-A,FSAC,14,False


### Update companies table

In [51]:
# db.clean_database()
# db._setup_database()

In [52]:
db.df_write(df_companies, "companies", commit=True)

# Stocks

In [66]:
def dfs_to_stocks(
    dfs: list[pd.DataFrame], prefix_to_market_id: dict, default_mid: int, is_pea: bool = False
) -> pd.DataFrame:
    df_all_days = pd.concat(dfs)
    #drop columns symbol and last_suffix
    df_all_days = df_all_days.drop(columns=["symbol", "last_suffix",'name'])
    return df_all_days

In [67]:
dfs = multi_read_df_from_paths(list(file_infos_df_filtered ["path"]))

  0%|          | 0/1944 [00:00<?, ?it/s]

100%|██████████| 1944/1944 [00:06<00:00, 289.65it/s]


In [68]:
df_stocks = dfs_to_stocks(dfs, db.prefix_to_market_id, default_mid=db.nasdaq_market_id) 

#### Show Nan last values

In [96]:
df_stocks[df_stocks['last'].isna()]

,last,volume,timestamp
symbol,,,


#### Show Nan volume values

In [81]:
df_stocks[df_stocks['volume'].isna()]

,last,volume,timestamp
symbol,,,


#### Show negative values for volume

In [82]:
df_stocks[df_stocks['volume'] < 0]

,last,volume,timestamp
symbol,,,
1rPCDA,16.60,-12475,2020-04-07 12:02:02
1rPCDA,16.72,-10871,2020-04-07 13:02:02
1rPCDA,16.94,-9815,2020-04-07 14:02:02
1rPCDA,16.68,-9298,2020-04-07 15:02:01
1rPCDA,16.74,-9280,2020-04-07 16:02:01
1rPCDA,16.88,-8366,2020-04-07 17:02:02


#### Show zero values for volume

In [97]:
df_stocks[df_stocks['volume'] == 0]

,last,volume,timestamp
symbol,,,
ABII,78.25,0,2019-01-15 09:02:02
ACCL,12.51,0,2019-01-15 09:02:02
ACTL,20.95,0,2019-01-15 09:02:02
ADGF,10.78,0,2019-01-15 09:02:02
AGAM,21.20,0,2019-01-15 09:02:02
...,...,...,...
1rPMLUMG,2.40,0,2023-12-21 17:02:02
1rPALUPG,1.80,0,2023-12-21 17:02:02
1rPMLVAL,14.30,0,2023-12-21 17:02:02


#### Show negatives values for last

In [98]:
df_stocks[df_stocks['last'] < 0]

,last,volume,timestamp
symbol,,,


#### Show zero values for last

In [99]:
df_stocks[df_stocks['last'] == 0]

,last,volume,timestamp
symbol,,,
ALZH,0.0,0,2019-01-15 09:02:02
AMCI,0.0,0,2019-01-15 09:02:02
ZLIG,0.0,0,2019-01-15 09:02:02
ASBC,0.0,0,2019-01-15 09:02:02
BECM,0.0,0,2019-01-15 09:02:02
...,...,...,...
1rPALGEC,0.0,14464536,2023-12-21 17:02:02
1rPMLGLA,0.0,0,2023-12-21 17:02:02
1rP03227,0.0,0,2023-12-21 17:02:02


#### Show max and min values for last

In [102]:
df_stocks[df_stocks['last'] == df_stocks['last'].max()].head(1)

,last,volume,timestamp
symbol,,,
1rPMALA,8750.0,0,2019-01-15 09:02:02


In [103]:
df_stocks[df_stocks['last'] == df_stocks['last'].min()].head(1)

,last,volume,timestamp
symbol,,,
ALZH,0.0,0,2019-01-15 09:02:02


#### Show the max and min value of the volume 

In [104]:
df_stocks[df_stocks['volume'] == df_stocks['volume'].max()].head()

,last,volume,timestamp
symbol,,,
1rEPALNEV,0.031,1234500128,2021-05-13 17:02:02


In [105]:
df_stocks[df_stocks['volume'] == df_stocks['volume'].min()].head()

,last,volume,timestamp
symbol,,,
1rPCDA,16.6,-12475,2020-04-07 12:02:02


In [110]:
### sort by volume 

df_stocks.sort_values(by="volume", ascending=False, inplace=True)
df_stocks.head(50)

,last,volume,timestamp
symbol,,,
1rEPALNEV,0.031,1234500128,2021-05-13 17:02:02
1rPALJXR,0.000,1216695504,2021-12-09 17:02:01
1rEPALNEV,0.002,1199764827,2021-10-05 17:02:01
1rPALJXR,0.000,1170246918,2021-12-09 16:02:01
1rEPALNEV,0.029,1167320278,2021-05-13 16:02:01
1rPALJXR,0.000,1139562259,2021-12-09 15:02:02
1rEPALNEV,0.002,1112984857,2021-10-05 16:02:01
1rEPALNEV,0.002,1085621107,2021-10-05 15:02:02
1rPALJXR,0.000,1062930783,2021-12-09 14:02:01


In [113]:
df_stocks = df_stocks[df_stocks['volume'] > 0]
df_stocks.sort_values(by="volume", ascending=True, inplace=True)
df_stocks

,last,volume,timestamp
symbol,,,
1rPSELER,77.500,1,2020-04-07 16:02:01
1rPALMII,4.040,1,2023-07-28 14:02:01
1rPALTHO,1.600,1,2023-04-03 09:02:01
1rPALEMG,4.060,1,2023-04-03 09:02:01
1rPALINT,1.240,1,2023-03-16 15:02:02
...,...,...,...
1rEPALNEV,0.029,1167320278,2021-05-13 16:02:01
1rPALJXR,0.000,1170246918,2021-12-09 16:02:01
1rEPALNEV,0.002,1199764827,2021-10-05 17:02:01
